In [1]:
import numpy as np
import pandas as pd 
import numpy.random as rn
from tqdm import tqdm_notebook as tqdm

from scipy import optimize       # to compare

/home/alice/Documents/ramax_vacation/venv/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [47]:
import pickle 

with open('possible_vacation.pickle', 'rb') as f:
    possible_vacation = pickle.load(f)

with open('req_work.pickle', 'rb') as f:
    req_work = pickle.load(f)
    
with open('excel_dict.pickle', 'rb') as f:
    excel_dict = pickle.load(f)

with open('qs.pickle', 'rb') as f:
    qs = pickle.load(f)

In [61]:
def annealing(random_start,
              cost_function,
              random_neighbour,
              acceptance,
              temperature,
              args,
              maxsteps=1000,
              debug=True
             ):
    state = random_start(*args)
    cost = cost_function(state)
    states, costs = [state], [cost]
    for step in range(maxsteps):
        fraction = step / float(maxsteps)
        T = temperature(fraction)
        new_state = random_neighbour(*args)  # fraction
        new_cost = cost_function(new_state)
#         if debug: print("Step #{:>2}/{:>2} : T = {:>4.3g}, state = {:>4.3g}, cost = {:>4.3g}, new_state = {:>4.3g}, new_cost = {:>4.3g} ...".format(step, maxsteps, T, state, cost, new_state, new_cost))
        if debug: print("Step #{:>2}/{:>2} : T = {:>4.3g} cost = {:>4.3g} new_cost = {:>4.3g} ...".format(step, maxsteps, T, cost, new_cost))          
        if acceptance_probability(cost, new_cost, T) > rn.random():
            state, cost = new_state, new_cost
            states.append(state)
            costs.append(cost)
    return state, cost_function(state), states, costs

In [50]:
def get_random_vac(max_fly_person, max_fly_month):
    return np.random.randint(36, min(max_fly_person, max_fly_month))


def random_matrix(df):
    state = np.zeros((df.shape[0], 12), dtype=int)
    max_fly_by_month = get_max_fly_by_month()
    aval_hours_by_month = get_D(df, by_month=True)

    for month in range(12):
        i = 0
        while aval_hours_by_month[month] > 0:
            i += 1
            if state.shape[0] - i < 0:
                break
            rand_i = np.random.randint(0, state.shape[0])

            _vac = get_random_vac(
                max_fly_person=df['Max Fly'].iloc[rand_i],
                max_fly_month=max_fly_by_month[month]
            )

            if state[rand_i][month] > 0:
                continue

            # Если уже использовали больше чем нужно отпуск
            if state[rand_i].sum() + _vac > max(df['Max Fly'].iloc[rand_i] * 2, 184):
                continue

            # Если у нас на расстоянии 2 уже есть отпуск
            if month < 11:
                if state[rand_i][month + 1] > 0 or state[rand_i][month - 1] > 0:
                    continue
            else:
                if state[rand_i][0] > 0 or state[rand_i][10] > 0:
                    continue

            # Если число доступных часов больше
            if aval_hours_by_month[month] >= _vac:
                aval_hours_by_month[month] -= _vac
                state[rand_i, month] = _vac

            # Если число доступных часов меньше, то переносим отпуск на след месяц
            else:
                delta = _vac - aval_hours_by_month[month]
                state[rand_i, month] = aval_hours_by_month[month]
                aval_hours_by_month[month] = 0

                if month < 11:
                    state[rand_i, month + 1] += delta
                    aval_hours_by_month[month + 1] -= delta
                else:
                    state[rand_i, 0] += delta
                    aval_hours_by_month[0] -= delta
                break
    return state

In [51]:
a = random_matrix(qs['Q9'])

In [52]:
# сделано
display(pd.Series(a.sum(axis=0)))

0      751
1     2677
2     1270
3     1768
4     2009
5      852
6      722
7      719
8      831
9     1247
10     590
11     684
dtype: int64

In [53]:
# надо
possible_vacation.loc['Q9', :]

1      751.0
2     2913.0
3     1270.0
4     1768.0
5     2009.0
6      852.0
7      722.0
8      719.0
9      831.0
10    1766.0
11    1108.0
12    1114.0
Name: Q9, dtype: float64

In [273]:
def get_max_fly_by_month():
    return excel_dict['months'][2].values

In [418]:
def get_D_d(vacs):
    return vacs.sum(axis=0)

def get_D(df, q_name, by_month=False):
    if not by_month:
        return possible_vacation.loc[q_name, :].sum()
    else:
        return possible_vacation.loc[q_name, :].values.copy()

def get_S(df):
    return df[[f'RP{i}_score' for i in range(1, 13)]].values.sum().sum()
    
def get_S_d(vacs, df):
    a = df[[f'RP{i}_score' for i in range(1, 13)]].values
    b = vacs.astype(bool).astype(int)
    return np.multiply(a, b).sum().sum()
    
def cost_function(vacs):
    return f(vacs)

In [297]:
def cost_function(vacs):
    cost = 1 - get_S_d(vacs, df) / get_S(df)
    return cost

In [56]:
def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        return 1
    else:
        p = np.exp(- (new_cost - cost) / temperature)
        return p

In [57]:
def temperature(fraction):
    return max(0.01, min(1, 1 - fraction))

In [ ]:
def output(q):

    l = []

    for vac, person, person_score in zip(state, df['Num'].values, df[[f'RP{i}_score' for i in range(1, 13)]].values):
        for i, (vac_size, score) in enumerate(zip(vac, person_score), 1):
            if vac_size != 0:
                l.append([person, i, vac_size, 1 if score else 0])
    
    _df = pd.DataFrame(data=l, columns=['id', 'month', 'rest_size', 'aplication'])
    _df.to_csv(f'../data/res_v.0.03_{q}.csv')

In [62]:
for q_name, df in qs.items():
    print(q_name)
    state, c, states, costs = annealing(
        random_matrix,
        cost_function,
        random_matrix,
        acceptance_probability,
        temperature,
        args=[df],
        maxsteps=1000,
        debug=True
    )
    

Q1
Step # 0/1000 : T =    1 cost = 0.937 new_cost = 0.951 ...
Step # 1/1000 : T = 0.999 cost = 0.951 new_cost = 0.954 ...
Step # 2/1000 : T = 0.998 cost = 0.954 new_cost = 0.941 ...
Step # 3/1000 : T = 0.997 cost = 0.941 new_cost = 0.948 ...
Step # 4/1000 : T = 0.996 cost = 0.948 new_cost = 0.937 ...
Step # 5/1000 : T = 0.995 cost = 0.937 new_cost = 0.939 ...
Step # 6/1000 : T = 0.994 cost = 0.939 new_cost = 0.938 ...
Step # 7/1000 : T = 0.993 cost = 0.938 new_cost = 0.931 ...
Step # 8/1000 : T = 0.992 cost = 0.931 new_cost = 0.94 ...
Step # 9/1000 : T = 0.991 cost = 0.94 new_cost = 0.946 ...
Step #10/1000 : T = 0.99 cost = 0.946 new_cost = 0.933 ...
Step #11/1000 : T = 0.989 cost = 0.933 new_cost = 0.951 ...
Step #12/1000 : T = 0.988 cost = 0.951 new_cost = 0.942 ...
Step #13/1000 : T = 0.987 cost = 0.942 new_cost = 0.936 ...
Step #14/1000 : T = 0.986 cost = 0.936 new_cost = 0.936 ...
Step #15/1000 : T = 0.985 cost = 0.936 new_cost = 0.951 ...
Step #16/1000 : T = 0.984 cost = 0.951 ne

Step #140/1000 : T = 0.86 cost = 0.942 new_cost = 0.93 ...
Step #141/1000 : T = 0.859 cost = 0.93 new_cost = 0.941 ...
Step #142/1000 : T = 0.858 cost = 0.941 new_cost = 0.945 ...
Step #143/1000 : T = 0.857 cost = 0.945 new_cost = 0.926 ...
Step #144/1000 : T = 0.856 cost = 0.926 new_cost = 0.938 ...
Step #145/1000 : T = 0.855 cost = 0.938 new_cost = 0.948 ...
Step #146/1000 : T = 0.854 cost = 0.948 new_cost = 0.939 ...
Step #147/1000 : T = 0.853 cost = 0.939 new_cost = 0.942 ...
Step #148/1000 : T = 0.852 cost = 0.942 new_cost = 0.943 ...
Step #149/1000 : T = 0.851 cost = 0.943 new_cost = 0.942 ...
Step #150/1000 : T = 0.85 cost = 0.942 new_cost = 0.931 ...
Step #151/1000 : T = 0.849 cost = 0.931 new_cost = 0.935 ...
Step #152/1000 : T = 0.848 cost = 0.935 new_cost = 0.952 ...
Step #153/1000 : T = 0.847 cost = 0.952 new_cost = 0.942 ...
Step #154/1000 : T = 0.846 cost = 0.942 new_cost = 0.957 ...
Step #155/1000 : T = 0.845 cost = 0.957 new_cost = 0.932 ...
Step #156/1000 : T = 0.844 c

KeyboardInterrupt: 

### Output